参考链接：https://foofish.net/python-closure.html


### 闭包（Closure ）

#### 闭包（closure）是函数式编程的重要的语法结构，Python也支持这一特性。
函数式编程是一种编程范式 (而面向过程编程和面向对象编程也都是编程范式)。
1. 在面向过程编程中，我们见到过函数(function)；
2. 在面向对象编程中，我们见过对象(object)。函数和对象的根本目的是以某种逻辑方式组织代码，并提高代码的可重复使用性(reusability)。

#### 嵌套函数
理解闭包之前，先区分一下嵌套函数。

函数不仅可以定义在模块的最外层，还可以定义在另外一个函数的内部，像这种定义在函数里面的函数称之为嵌套函数（nested function）。

例如：

In [1]:
def print_msg():
    # print_msg 是外围函数
    msg = "zen of python"

    def printer():
        # printer是嵌套函数
        print(msg)
    printer()
    
print_msg()

zen of python


对于嵌套函数，它可以访问到其外层作用域中声明的非局部（non-local）变量，比如代码示例中的变量 msg 可以被嵌套函数 printer 正常访问。

作用域之外我们无法使用局部变量。

In [2]:
msg

NameError: name 'msg' is not defined

那么有没有一种可能即使脱离了函数本身的作用范围，局部变量还可以被访问得到呢(函数内的局部变量，但是作用域不是局部的，而且不是全局变量)？答案是闭包

In [3]:
def print_msg():
    # print_msg 是外围函数
    msg = "zen of python"
    def printer():
        # printer 是嵌套函数
        print(msg)
    return printer

another = print_msg()
print(another)

<function print_msg.<locals>.printer at 0x00000000047FED08>


调用print_msg()时，返回的并不是求和结果，而是打印函数。

another 对象指向了函数print_msg，another()才是指向目标函数。

In [4]:
another()

zen of python


这段代码和前面例子的效果完全一样，同样输出 "zen of python"。不同的地方在于内部函数 printer 直接作为返回值返回了。

一般情况下，函数中的局部变量仅在函数的执行期间可用，一旦 print_msg() 执行过后，我们会认为 msg变量将不再可用。然而，在这里我们发现 print_msg 执行完之后，在调用 another 的时候 msg 变量的值正常输出了，这就是闭包的作用，闭包使得局部变量在函数外被访问成为可能。

**这里的 another 就是一个闭包，闭包本质上是一个函数**，它有两部分组成，printer 函数和变量 msg。**闭包使得这些变量的值始终保存在内存中**，生命周期被加长，而不仅仅做用于局部函数中。

闭包，顾名思义，就是一个封闭的包裹，里面包裹着自由变量，就像在类里面定义的属性值一样，自由变量的可见范围随同包裹，哪里可以访问到这个包裹，哪里就可以访问到这个自由变量。

In [5]:
def adder(x):
    def wrapper(y):
        return x + y
    return wrapper

adder5 = adder(5)

In [6]:
adder5(10)

15

In [7]:
adder5(6)

11

在adder5 = adder(5)这个表达式中，adder(5)--> x = 5，因为在adder()函数中 return wrapper，所以 y = 5。

adder5(10)相当于执行adder.``<locals>``.wrapper(10)，最后 return x + y --> 10 + 5

所有函数都有一个 ``__closure__``属性，如果这个函数是一个闭包的话，那么它返回的是一个由 cell 对象 组成的元组对象。cell 对象的cell_contents 属性就是闭包中的自由变量(adder()函数中的y)。

In [8]:
adder.__closure__

In [9]:
adder5.__closure__

(<cell at 0x00000000047EABB8: int object at 0x0000000054BAC8C0>,)

In [10]:
adder5.__closure__[0].cell_contents

5

这解释了为什么局部变量脱离函数之后，还可以在函数之外被访问的原因的，因为它存储在了闭包的 cell_contents中了。

#### 当一个函数返回了一个函数后，其内部的局部变量还被新函数引用，例如上面例子中，函数lazy_sum中又定义了函数sum，并且，内部函数sum可以引用外部函数lazy_sum的参数和局部变量，当lazy_sum返回函数sum时，相关参数和变量都保存在返回的函数中。<br><br>

### 先解释一下。变量绑定的问题

In [11]:
def list_a():
    return [1, 2, 3]

l = list_a()
l

[1, 2, 3]

In [12]:
a, b ,c = list_a()
print(a)
print(b)
print(c)

1
2
3


#### 上面的例子中a, b, c是普通变量。a, b, c 的值依次为list中的三个元素。list_a()是一个普通函数，返回值为一个集合。
再来看Python教程中的例子

In [13]:
def count():
    fs = []
    for i in range(1, 4):
        def f():
            return i * i# f()没有传入形参i，而是直接引用外部变量i
        fs.append(f)
    return fs

f1, f2, f3 = count()# 注意f1, f2, f3是闭包
print(count())
print(f1())
print(f2())
print(f3())

[<function count.<locals>.f at 0x000000000488C378>, <function count.<locals>.f at 0x000000000488C268>, <function count.<locals>.f at 0x000000000488C1E0>]
9
9
9


#### 为什么f1(), f2(), f3() 返回结果为9， 9， 9。而不是1， 4， 9。
对于这个函数count()，我们在定义f()时，f()里面的i和range(1, 4)绑定，可以理解为i --> [1, 2, 3]，此时i要想有意义地被使用，不能再是一个集合，必须是**一个具体的数值**才能执行过程i * i，而i会最终绑定到集合的最末元素，__因此最终结果为9__。

如果一定要引用循环变量怎么办？方法是再创建一个函数，用该函数的参数绑定循环变量当前的值，无论该循环变量后续如何更改，已绑定到函数参数的值不变：

In [14]:
def count():
    fs = []
    def f(n):# 传入形参
        def j():
            return n * n
        return j
    for i in range(1, 4):
        fs.append(f(i))
    return fs

f1, f2, f3 = count()

print(f1())
print(f2())
print(f3())

1
4
9


实际上调用count()函数，会返回一个集合，这个集合包含是三个元素，每个元素对应的都是一个函数f(i)。

这是为什么呢？

因为在count()函数中显然f(n)里面的n*n已经被绑定,明确传入了形参n，调用f(n)时传入的参数n的值来决定，而不是依赖于range(1,4)。

此时count()函数是一个嵌套了一个闭包函数的**普通函数**，返回了一个有3个元素的集合。f()函数才是一个闭包函数。